In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
train_data = pd.read_csv("drive/MyDrive/Colab-Notebooks/train.csv")
test_data = pd.read_csv("drive/MyDrive/Colab-Notebooks/test.csv")
test_ids = test_data["PassengerId"]

In [ ]:
def clean(data):
    data = data.drop(["PassengerId", "Name", "Ticket"], axis=1)

    cols = ["SibSp", "Parch", "Fare", "Age"]
    for col in cols:
        data[col].fillna(data[col].mean(), inplace=True)

    data.Embarked.fillna("U", inplace=True)

    return data

In [ ]:
train_data = clean(train_data)
test_data = clean(test_data)

In [ ]:
train_data.head(5)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,0,3,male,22.0,1,0,7.2500,NaN,S
1,1,1,female,38.0,1,0,71.2833,C85,C
2,1,3,female,26.0,0,0,7.9250,NaN,S
3,1,1,female,35.0,1,0,53.1000,C123,S
4,0,3,male,35.0,0,0,8.0500,NaN,S


In [ ]:
X = train_data.iloc[:, 1:]
Y = train_data.iloc[:, 0]
print(f"train data shape: {X.shape}")
print(f"test data shape: {test_data.shape}")

train data shape: (891, 8)
test data shape: (418, 8)


In [ ]:
dummy_x = pd.get_dummies(X)
print(dummy_x.shape)
dummy_x

(891, 158)


,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Cabin_A10,Cabin_A14,Cabin_A16,...,Cabin_F2,Cabin_F33,Cabin_F38,Cabin_F4,Cabin_G6,Cabin_T,Embarked_C,Embarked_Q,Embarked_S,Embarked_U
0,3,22.000000,1,0,7.2500,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,38.000000,1,0,71.2833,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,3,26.000000,0,0,7.9250,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1,35.000000,1,0,53.1000,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,3,35.000000,0,0,8.0500,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,2,27.000000,0,0,13.0000,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
887,1,19.000000,0,0,30.0000,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
888,3,29.699118,1,2,23.4500,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
889,1,26.000000,0,0,30.0000,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
sc_x = scaler.fit_transform(dummy_x)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

In [ ]:
depth = []
for i in tqdm(range(2,50,1)):
  rf = RandomForestClassifier(max_depth=i)
  scores = cross_val_score(rf, sc_x, Y, cv=5)
  depth.append(scores.mean())

100%|██████████| 48/48 [00:54<00:00,  1.13s/it]


In [ ]:
final_depth = np.argmax(np.array(depth)) + 2
print(final_depth)

18


In [ ]:
estimators = []
for i in tqdm(range(50,200,1)):
  rf = RandomForestClassifier(n_estimators=i, max_depth=22)
  scores = cross_val_score(rf, sc_x, Y, cv=5)
  estimators.append(scores.mean())

100%|██████████| 150/150 [03:34<00:00,  1.43s/it]


In [ ]:
final_estimators = np.argmax(np.array(estimators)) + 50
print(final_estimators)

57


In [22]:
final_rf = RandomForestClassifier(n_estimators=final_estimators, max_depth=final_depth)
final_rf.fit(sc_x, Y)

RandomForestClassifier(max_depth=18, n_estimators=57)

In [23]:
dummy_test_data = pd.get_dummies(test_data)
sc_test_data_x = scaler.transform(dummy_test_data)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- Cabin_A11
- Cabin_A18
- Cabin_A21
- Cabin_A29
- Cabin_A9
- ...
Feature names seen at fit time, yet now missing:
- Cabin_A10
- Cabin_A14
- Cabin_A16
- Cabin_A19
- Cabin_A20
- ...

  warnings.warn(message, FutureWarning)


ValueError: ignored

In [ ]:
submission = final_rf.predict(sc_test_data_x)
print(submission)

NameError: ignored

In [ ]:
submission_data = pd.DataFrame({"PassengerId": test_ids.values, "Survived": submission })
submission_data.to_csv("submission.csv", index=False)

NameError: ignored